In [3]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import time
import pandas as pd
import tabulate
import numpy as np
from IPython.core.display import display, HTML, clear_output
import functools
from collections import defaultdict
import parmap
import matplotlib.pyplot as plt
import math
import _pickle as cPickle
import copy
import warnings
import matplotlib
from matplotlib import rc, rcParams
import matplotlib.font_manager as fm

warnings.filterwarnings('ignore')

rcParams['font.family'] = 'Linux Libertine O'
rcParams["font.weight"] = "normal"
#customize fonts
my_font = fm.FontProperties(fname='/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf/LinLibertine_R.otf')
my_font.get_name()

'Linux Libertine O'

In [4]:
### prepare/clear data omitted

In [ ]:
#ax1 --- overall barplot; ax2 --- overall boxplot
#ax3 --- barplots by recipients ;ax4 --- boxplots by recipients

#height ratios should be adjusted based on the number of rows, currently hard-coded 1:15
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex='col', #sharey='row', \
figsize=(12,8),gridspec_kw={'height_ratios': [1, 15], 'width_ratios':[3, 1]})
plt.subplots_adjust(wspace=0.0, hspace=0)
ax1.set_xlim((0,1))
ax3.set_xlim((0,1))

colors_likert_red=["#005ab5",'#97a4d7','#f7f7f7','#f99c7d','#dc3200'][::-1]
label_list= ['Unacceptable','Somewhat unacceptable','Neither','Somewhat acceptable','Acceptable']


#prepare data and dataframe data --- table1_df
table1_df = table1_columns.melt(var_name='place', value_name='level').dropna()
feature, dependent_var="place", "level"
selected_columns = table1_df.groupby([feature, dependent_var])
unstack_columns =  selected_columns[feature].count().unstack(dependent_var)
percentage = selected_columns[feature].count()/unstack_columns.sum(axis=1)
temp = percentage.unstack(dependent_var).sort_values(by=[1,2,3,4,5], ascending=True)
g = temp.plot(ax=ax3, kind='barh', stacked=True, color=colors_likert_red[::-1], width=0.7)
font = fm.FontProperties(family='Linux Libertine O',style='normal', size=12)
g.legend(label_list[::-1], loc='upper left',
         ncol=5,fontsize='large',bbox_to_anchor=(-0.46, 0.15, 0, 1), prop=font)
#anchor box position hard-coded

#prepare barplot data omitted --- np_data
np_data = np.array(result, dtype=object)
bp1 = ax4.boxplot(np_data, \
                  sym='*', widths=0.7, patch_artist=True, showfliers=True, vert=False)
all_data = []
for l in np_data:
    all_data.extend(l)
    
bp_overall = ax2.boxplot(all_data,\
                  sym='*', widths=0.7, patch_artist=True, showfliers=True, vert=False)


### customize boxplot indicator colors
boxplot_facecolor = '#ffdc3c'#'#F9DB22'
boxplot_wirecolor = '#ffaa28'#"#A4CC44"
boxplot_mediancolor = '#f07814'#'#238823'#"#89AB0D"

for me in bp1['medians']:
    me.set(color=boxplot_mediancolor, linewidth=5)

for box in bp1['boxes']:
    box.set(color=boxplot_wirecolor, linewidth=2)
    box.set(facecolor = boxplot_facecolor )
    
for me in bp_overall['medians']:
    me.set(color=boxplot_mediancolor, linewidth=5)

for box in bp_overall['boxes']:
    box.set(color=boxplot_wirecolor, linewidth=2)
    box.set(facecolor = boxplot_facecolor )


### ax1 ax3 (barplots)
### add paddings to long text labels 
for tick in ax3.yaxis.get_majorticklabels():
    tick.set_horizontalalignment("right")
    tick.set_verticalalignment("center")

ax3.tick_params(axis = 'y', which = 'major', labelsize = 18)
ax1.get_yaxis().set_tick_params(pad=10)


### add vertical gray lines spaced every 20%
for itvl in np.arange(0, 1.01,0.2):
    ax1.axvline(x=itvl, linestyle='--',color="gray",linewidth=0.8)
    
for itvl in np.arange(0, 1.01,0.2):
    ax3.axvline(x=itvl, linestyle='--',color="gray",linewidth=1.2)

### -------- graph ax1 barplot Overall ------------
ax1.set_ylabel("Overall",rotation='horizontal',fontsize=20,y=0.1,labelpad=20)
overall = unstack_columns.sum(axis=0)
overall_percentage = overall/overall.sum()
overall_percentage_df = overall_percentage.to_frame().transpose()
g_overall = overall_percentage_df.plot(ax=ax1,kind='barh', stacked=True, color=colors_likert_red[::-1], width=0.9)
ax1.get_legend().remove()
ax1.set_yticklabels([])
    
#ax1 ----- annotate------ barplot Overall   
for j, p in enumerate(ax1.patches):
    if (j==0):
        color='#FFFFFF'
    else:
        color='black'
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy() 
    txt = str(int(np.round(width*100, decimals=0)))
    ax1.annotate(txt+'%', (x+0.5*width, y + 0.4*height), \
                ha="center", va="center",size=16, color=color)


### ------- ax3 x.y labels -----------------------
ax3.set_xticklabels(['0%', '20%', '40%', '60%', '80%', '100%'], fontsize=15)
ax3.set_ylabel("Recipients", rotation=90, size=20)    
#ax3.set_yticklabels([])
#ax3.set_xticklabels(fontsize=15)


### ------- annotate each bar segment with percentage text
for j, p in enumerate(ax3.patches):
    if (j//21==0) or (j//21==0): #change color to make texts more visible
        color='#FFFFFF'
    else:
        color='black'
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy()
    txt = str(int(np.round(width*100, decimals=0)))
    #anot = txt.astype('str')

    ax3.annotate(txt+'%', (x+0.5*width, y + 0.4*height), \
                ha="center", va="center",size=16, color=color)

    
#------barplots ax2 & ax4 labeling--------------
ax2.set_yticklabels([])
ax4.set_yticklabels([])

font = fm.FontProperties(family='Linux Libertine O',style='normal', size=12)

ax_shared=ax2.twiny()
ax_shared.xaxis.set_ticks_position('top')
ax_shared.set_xticklabels(['Acceptable', "", "Neither", "", "Unacceptable"], \
                          rotation=0, fontproperties=font) #only use three labels due to limited space

ax_shared.set_xlim([0.85,5.15])
ax_shared.set_xticks([1,2,3,4,5])

ticks_font = fm.FontProperties(family='Linux Libertine O',
                                   style='normal', size=13)
for label in ax_shared.xaxis.get_majorticklabels():
    label.set_fontproperties(ticks_font)

ax_shared.set_ylim([0.5,1.5])
ax_shared.xaxis.tick_top()
ax4.tick_params(labelbottom=False)

#save     
plt.savefig('plot.pdf', format='pdf', dpi=fig.dpi, bbox_inches='tight')